<a href="https://colab.research.google.com/github/anindita220492/smart_agribot/blob/main/Smart_AgriBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section A: Setup Gemini API

In [1]:
#Install Vertex AI SDK
!pip install google-cloud-aiplatform --quiet

In [2]:
# Authenticate with my Google Cloud Account
from google.colab import auth
auth.authenticate_user()

In [3]:
#Initialize Vertex AI
import vertexai
from vertexai.language_models import ChatModel

PROJECT_ID = "clean-doodad-460707-p3"
vertexai.init(project=PROJECT_ID, location="us-central1")

In [4]:
#Ask Gemini a question
from vertexai.preview.generative_models import GenerativeModel

model = GenerativeModel("gemini-2.0-flash-001")
response = model.generate_content("What are the common diseases in tomato plant?")
print (response.text)

Tomato plants are susceptible to a wide variety of diseases, both fungal, bacterial, and viral. Here's a breakdown of some of the most common ones:

**Fungal Diseases:**

*   **Early Blight (Alternaria solani):**
    *   **Symptoms:** Dark brown spots with concentric rings (bullseye pattern) appear on lower leaves, gradually moving upwards.  Fruit can also develop similar spots, often near the stem end.
    *   **Conditions:** Warm, humid weather, often after periods of rain or overhead watering.
    *   **Control:**  Good sanitation (remove infected leaves), crop rotation, proper spacing for air circulation, fungicide applications (copper-based, chlorothalonil, mancozeb).

*   **Late Blight (Phytophthora infestans):**
    *   **Symptoms:** Irregular, water-soaked lesions on leaves, stems, and fruit.  A white, fuzzy growth may appear on the undersides of leaves in humid conditions. Fruit turns brown, leathery, and decays rapidly.  Spreads very quickly.
    *   **Conditions:** Cool, wet

**Section B: Simulate Diseases Detection**

In [6]:
#Running cell to upload files
from google.colab import files
uploaded = files.upload()

Saving sample_crop_data.txt to sample_crop_data.txt


In [9]:
def detect_disease_mock(sample_crop_data):
  if "leaf" in sample_crop_data.lower():
    return "Powdery Mildew - Use sulfue based fungicide."
  else:
    return "Unable to detect diseases. Try another image."
print(detect_disease_mock("tomato_leaf_spot.jpg")) # Corrected the function call

Powdery Mildew - Use sulfue based fungicide.


**Section C: Fetch Weather Data**

In [10]:
API_KEY = "48493aec6dae41d0b5b32002acd9546"

In [11]:
import requests

def get_weather(city, api_key=API_KEY):
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {"key": api_key, "q": city}
    response = requests.get(base_url, params=params)
    data = response.json()

In [12]:
import requests

def get_weather(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if "cod" in data and data["cod"] != 200:
        print(f"Error fetching weather data: {data['message']}")
        return None, None

    weather = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    return weather, temp

# Example usage
API_KEY = "48493aec6dae41d0b5b32002acd95463"  # Replace with your actual key
weather, temp = get_weather("Chandannagar", API_KEY)
print("Weather:", weather)
print("Temperature:", temp)

Weather: broken clouds
Temperature: 32.94


Upto this, I have built
1. A workable Gemini chatbot
2. A diseases dector mock
3. A weather lookup






**Step 2: Build the Streamlit App (Frontend)**

In [13]:
#utils/gemini_app.py
!mkdir utils

with open("utils/gemini_app.py", "w") as f:
  f.write("""

from vertexai.language_models import ChatModel
import vertexai

def ask_gemini(question: str, project_id: str, location: str = "us-central1") -> str:
    vertexai.init(project=project_id, location=location)
    chat_model = ChatModel.from_pretrained("chat-bison")
    chat = chat_model.start_chat()
    response = chat.send_message(question)
    return response.text
""")


In [14]:
#utils/diseases_classifier.py
with open("utils/disease_classifier.py", "w") as f:
    f.write("""
def detect_disease(image_file):
    if "leaf" in image_file.name.lower():
        return "Powdery Mildew - Use sulfur-based fungicide."
    return "Unable to detect disease. Please upload a clearer image."
""")

In [15]:
# utils/weather_api.py
with open("utils/weather_api.py", "w") as f:
    f.write("""
import requests

def get_weather(city, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url).json()
        weather = response['weather'][0]['description']
        temp = response['main']['temp']
        return weather, temp
    except:
        return "Error", "N/A"
""")

In [17]:
%%writefile streamlit_app.py


#!pip install streamlit --quiet

import streamlit as st
from utils.gemini_app import ask_gemini
from utils.disease_classifier import detect_disease
from utils.weather_api import get_weather

PROJECT_ID = st.secrets["PROJECT_ID"]
WEATHER_API_KEY = st.secrets["WEATHER_API_KEY"]

st.set_page_config(page_title="Smart AgriBot", layout="wide")
st.title("🌾 Smart AgriBot: AI Assistant for Small Farmers")

tab1, tab2, tab3 = st.tabs(["💬 Ask AgriBot", "🧪 Diagnose Disease", "🌦️ Weather Info"])

with tab1:
    query = st.text_input("Ask your agricultural question (in any language):")
    if st.button("Get Answer") and query:
        response = ask_gemini(query, project_id=PROJECT_ID)
        st.success(response)

with tab2:
    st.write("Upload an image of a diseased plant leaf:")
    uploaded_image = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
    if uploaded_image:
        result = detect_disease(uploaded_image)
        st.info(f"Disease Diagnosis: {result}")
        st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

with tab3:
    city = st.text_input("Enter your city/village:")
    if st.button("Check Weather") and city:
        weather, temp = get_weather(city, api_key=WEATHER_API_KEY)
        st.write(f"🌤️ Weather: {weather}")
        st.write(f"🌡️ Temperature: {temp}°C")

Overwriting streamlit_app.py
